# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [3]:
infile = f"../data/sents_2.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [4]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [5]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [6]:
len(bigram_phrases.vocab.keys())

5978799

Prenons une clé au hasard :

In [7]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

absence_&


Le dictionnaire indique le score de cette coocurrence :

In [8]:
bigram_phrases.vocab[key_]

1

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

In [9]:
%time bigram_phrases[sentences]

CPU times: user 1.33 ms, sys: 6 µs, total: 1.34 ms
Wall time: 5.16 ms


### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [10]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

In [11]:
%time bigram_phraser[sentences]

CPU times: user 3.56 ms, sys: 15 µs, total: 3.57 ms
Wall time: 3.49 ms


### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [12]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [13]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [14]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [15]:
print(corpus[:100])

[['v', 'i', 'l', 'l', 'e', 'de', 'bruxelles', 'bulletin', 'ires', '8eanas', 'dl', '!'], ['conseil_communal', 'annee', '1847', '.'], ['au', 'ville', 'de', 'b', 'r', 'u', 'x', 'e', 'l', 'l', 'e', 's', '.'], ['bulletin', 'conseil', 'aes', 'seances', 'communal', '.'], ['annee', '1847', '.'], ['bruxelles', ',', 'imprimerie', 'd', 'e', 'j', '.'], ['h', '.', 'b', 'r', 'i', 'a', 'r', 'd', ',', 'rite', 'n', 'e', 'u', 'v', 'e', ',', '3', '1', ',', 'faubourg', 'de', 'n', 'a', 'm', 'u', 'r', ',', '1', '84', '8', 'de', '!'], ['du', 'consei', 'dibi', 'e', '.', '-', 'communication', 'conclusions', 'de', 'la', 'section', 'des', 'du', 'nouvel_hospice', 'pour', 'les', 'av', 'enraisonde', 'l', "'", 'absence', '&', 'maladie', '.', 'le', 'conseil', 'ajourne', 'leurs', 'de', 'pierre', 'el', 'marchai', 'cles', 'des', 'taxes', 'communale', "'", 'bieniaance', 'eldeseianv', 'il', 'est', 'donne', 'communie', ';', 'mandant', 'le', 'o', 'p', 'fa', 'gnant', 'l', "'", 'envoi', 'de', 'leur', 'bn', 'par', 'l', "'", 'e

## Entrainement d'un modèle Word2Vec sur ce corpus (modèle 2)

In [16]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 19min 5s, sys: 47.3 s, total: 19min 52s
Wall time: 7min 21s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [17]:
outfile = f"../data/bulletins_2.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [28]:
model = Word2Vec.load("../data/bulletins_2.model")

### Imprimer le vecteur d'un terme

In [29]:
model.wv["bruxelles"]

array([ 0.58973897, -1.0262196 ,  2.0716813 ,  1.589865  ,  1.3657871 ,
       -2.0293982 , -0.21928205,  4.652348  , -4.202351  , -4.3337784 ,
       -0.86379164,  5.786408  , -2.7091334 ,  1.221566  ,  3.5432715 ,
        0.9311423 , -1.8224436 ,  1.488655  ,  2.5310202 , -0.78291976,
        0.30947435, -3.211929  , -1.6163418 ,  4.165647  , -3.4496503 ,
        0.3898385 ,  0.67458636,  1.77817   ,  1.7902862 ,  2.8616135 ,
       -3.8638105 , -0.1167161 ], dtype=float32)

### Calculer la similarité entre deux termes

In [30]:
model.wv.similarity("boucher", "boulanger")

0.86908996

### Chercher les mots les plus proches d'un terme donné

In [31]:
model.wv.most_similar("bruxelles", topn=10)

[('laeken', 0.8239486217498779),
 ('considere_comme_officiel', 0.7925914525985718),
 ('liege', 0.7702558040618896),
 ('paris', 0.7679300308227539),
 ('nivelles', 0.7521665096282959),
 ('gand', 0.7397889494895935),
 ('lille', 0.7102484703063965),
 ('differents_quartiers', 0.6752375960350037),
 ('lille_1860', 0.6710933446884155),
 ('louvain', 0.6700588464736938)]

### Faire des recherches complexes à travers l'espace vectoriel

In [32]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('lille', 0.8239251375198364), ('ostende', 0.8021267056465149), ('gand', 0.7774108052253723), ('manchester', 0.7756398320198059), ('londres', 0.7693049311637878), ('tournai', 0.7622190117835999), ('verviers', 0.7603538036346436), ('paris', 0.7556376457214355), ('louvain', 0.7518820762634277), ('prague', 0.7503906488418579)]


### Modèle 3: Taille du vecteur = 100 (vs 32)

In [33]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=100, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 26min 12s, sys: 1min 6s, total: 27min 19s
Wall time: 9min 32s


In [34]:
outfile = f"../data/bulletins_3.model"
model.save(outfile)

In [35]:
model = Word2Vec.load("../data/bulletins_3.model")

In [36]:
model.wv["bruxelles"]

array([-2.821787  ,  1.7417598 , -0.47517857, -0.9297561 ,  0.20933826,
       -0.33410355,  1.0013304 , -3.9408467 ,  2.2906547 , -1.2519214 ,
        2.0284204 ,  0.4734065 ,  0.7797554 , -2.3764455 ,  2.3031328 ,
        1.3654215 , -0.23531204,  4.263294  ,  2.5052593 ,  1.7040656 ,
        0.7755428 ,  1.1707009 , -3.485699  , -1.4181716 ,  1.7519428 ,
       -1.1885886 , -0.9323578 ,  2.7376783 , -0.05357512,  3.0506134 ,
       -0.04246143,  3.1094618 ,  2.361703  ,  0.3488069 ,  0.13668808,
        1.7061647 ,  1.0303104 ,  0.16420278, -2.3176546 , -1.2321485 ,
        1.0473685 , -0.16010597,  0.21304561, -3.2333903 ,  1.7420102 ,
       -1.5983607 , -1.4976974 ,  0.67025024,  2.446454  ,  2.8469923 ,
       -0.8682723 ,  0.5673862 , -1.9958344 ,  0.12715669, -2.2643695 ,
       -0.83515686,  0.38899907, -2.7805963 ,  1.0056809 ,  2.4225903 ,
       -0.6718316 ,  0.7441178 , -0.16049749,  1.4168892 ,  2.6510496 ,
        0.04205501, -4.032287  ,  4.0189724 , -2.0547545 , -2.65

In [37]:
model.wv.similarity("boucher", "boulanger")

0.7806577

In [38]:
model.wv.most_similar("bruxelles", topn=10)

[('rruxelles', 0.6470252275466919),
 ('paris', 0.6332045793533325),
 ('liege', 0.631273090839386),
 ('nivelles', 0.6280113458633423),
 ('gand', 0.6024729013442993),
 ('laeken', 0.5872405767440796),
 ('lille', 0.5851026773452759),
 ('bruxelle', 0.5699034929275513),
 ('jumeaux_naissances', 0.5477592945098877),
 ('malines', 0.5462380647659302)]

In [39]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('lille', 0.6613479256629944), ('paris', 0.6434320211410522), ('verviers', 0.6389840841293335), ('londres', 0.6370319724082947), ('hambourg', 0.6282646059989929), ('prague', 0.6222774386405945), ('gand', 0.6142219305038452), ('tournai', 0.6134740114212036), ('liege', 0.6133009791374207), ('ostende', 0.6108174920082092)]


### Modèle 4: Vecteur 100 + élargissement de la fenêtre à 20 (vs 5)

In [40]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=100, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=20, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 31min 2s, sys: 1min 10s, total: 32min 12s
Wall time: 10min 3s


In [41]:
outfile = f"../data/bulletins_4.model"
model.save(outfile)

In [117]:
model = Word2Vec.load("../data/bulletins_4.model")

In [118]:
model.wv["bruxelles"]

array([-3.5294814 ,  0.46205315, -3.136256  ,  2.7181191 , -0.19754034,
        1.3294256 , -5.4065156 , -1.7076005 ,  1.1482389 , -1.7121512 ,
        3.2751715 , -6.316165  , -2.3904014 ,  2.6055224 , -4.316526  ,
       -1.3751478 ,  3.7133205 , -0.11183983,  2.466617  , -2.2758758 ,
        6.209551  , -5.186154  ,  2.6362963 ,  4.2062054 ,  1.7750355 ,
       -2.3785162 , -2.730668  , -0.01242101, -4.154819  , -0.3899968 ,
       -0.77142626,  0.82222265, -3.6569283 , -1.4491752 ,  4.992444  ,
       -0.6587832 ,  1.5070369 , -0.5447458 ,  2.6052966 ,  0.9000388 ,
       -3.480406  ,  0.1302352 ,  2.097107  , -3.139999  ,  0.8931701 ,
       -4.249983  , -0.11780771,  1.2884921 ,  3.125195  , -3.7342858 ,
       -6.842377  , -0.7513382 ,  0.7284325 ,  0.5234136 , -1.0563607 ,
       -7.118118  ,  5.3821054 , -4.446206  ,  3.4555452 , -1.8658125 ,
        2.457817  ,  1.0376217 ,  2.5419545 , -3.8404133 ,  2.0115778 ,
       -1.5690126 , -0.6278956 ,  1.6097066 ,  2.5003555 , -0.84

In [119]:
model.wv.similarity("boucher", "boulanger")

0.69779754

In [120]:
model.wv.most_similar("bruxelles", topn=10)

[('xelles', 0.6508788466453552),
 ('dette_active', 0.6217373609542847),
 ('bruxelle', 0.5955502390861511),
 ('liege', 0.586171567440033),
 ('laeken', 0.5658113956451416),
 ('nivelles', 0.559372067451477),
 ('hutel', 0.5444490909576416),
 ('vhotel', 0.5254800915718079),
 ('manufacturiere', 0.5184491276741028),
 ('bruxel', 0.5167596340179443)]

In [121]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('liege', 0.6075229048728943), ('manchester', 0.6038661599159241), ('hambourg', 0.5692122578620911), ('ostende', 0.5674587488174438), ('paris', 0.5573484897613525), ('lille', 0.5570899844169617), ('cologne', 0.5560243129730225), ('gand', 0.5556208491325378), ('amsterdam', 0.5449244976043701), ('tournai', 0.5420806407928467)]


In [123]:
print(model.wv.most_similar(positive=['homme', 'roi'], negative=['femme']))

[('homme_politique', 0.6000778079032898), ('souverain', 0.5965874195098877), ('episode', 0.5854206085205078), ('defenseur', 0.5852083563804626), ('peuple', 0.5814440250396729), ('prince', 0.5742866396903992), ('parlement', 0.5693864822387695), ('magistrat', 0.5646989345550537), ('congres', 0.564418375492096), ('ecria', 0.562084972858429)]


In [124]:
print(model.wv.most_similar(positive=['france', 'belgique'], negative=['paris']))

[('russie', 0.6767497658729553), ('italie', 0.665309727191925), ('afrique', 0.6608356833457947), ('allemagne', 0.6561207175254822), ('grece', 0.6292099952697754), ('egypte', 0.6222110390663147), ('republique', 0.6180391311645508), ('nation', 0.6069801449775696), ('autriche', 0.6021126508712769), ('hollande', 0.598942756652832)]


### Modèle 5: Taille fenêtre 20 (vs 5) et vecteur 32 (vs 100)

In [47]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=20, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 26min 43s, sys: 1min 14s, total: 27min 58s
Wall time: 9min 51s


In [48]:
outfile = f"../data/bulletins_5.model"
model.save(outfile)

In [49]:
model = Word2Vec.load("../data/bulletins_5.model")

In [50]:
model.wv["bruxelles"]

array([-0.503185  , -0.01135701,  2.9071445 , -0.33935323,  0.7852202 ,
       -7.8717766 , -1.3153803 ,  8.027404  , -1.2895185 , -9.461477  ,
        0.55994606,  7.050791  , -0.3875398 , -0.6959667 ,  3.3068764 ,
       -7.1694727 , -3.735939  ,  6.395983  ,  2.3281946 ,  2.3793006 ,
       -4.2038946 , -7.604685  , -0.70219207,  4.476044  , -0.3055644 ,
       -0.7566809 , -1.6676246 , -1.3390484 , -2.741731  ,  4.5634756 ,
        0.02120536,  0.61037207], dtype=float32)

In [51]:
model.wv.similarity("boucher", "boulanger")

0.8505966

In [52]:
model.wv.most_similar("bruxelles", topn=10)

[('laeken', 0.8055386543273926),
 ('liege', 0.7914955615997314),
 ('gand', 0.7288044095039368),
 ('bruxelloise', 0.7005123496055603),
 ('aux_confins', 0.6939319372177124),
 ('nivelles', 0.6921486854553223),
 ('obbgations_tramways', 0.6838479042053223),
 ('ostende', 0.6827777624130249),
 ('paris', 0.6722395420074463),
 ('louvain', 0.6675084829330444)]

In [53]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('liege', 0.8014519214630127), ('huy', 0.7924149632453918), ('gand', 0.7921144962310791), ('ostende', 0.7839378714561462), ('hambourg', 0.7556935548782349), ('manchester', 0.7534753680229187), ('lille', 0.7480643391609192), ('verviers', 0.7413298487663269), ('jadis', 0.7364925146102905), ('berlin', 0.735415518283844)]


### Modèle 6 : Vecteur 300 et fenêtre 10

In [54]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=300, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=10, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 52min 11s, sys: 2min 14s, total: 54min 25s
Wall time: 16min 7s


In [55]:
outfile = f"../data/bulletins_6.model"
model.save(outfile)

In [56]:
model = Word2Vec.load("../data/bulletins_6.model")

In [57]:
model.wv["bruxelles"]

array([-1.8941535 , -1.8441689 , -1.2983671 , -0.47930893, -1.4705082 ,
        1.243803  ,  0.12675878,  0.46231017, -1.1026258 ,  0.11179759,
        0.067589  ,  0.2038403 , -1.3499236 ,  4.0949454 , -0.36215502,
       -1.1563224 ,  0.82115173,  0.82290274, -0.9631553 , -0.68517464,
       -3.4759614 , -1.2089055 ,  1.694218  ,  1.6561887 , -1.1776484 ,
       -0.30590585, -1.2849115 , -0.8244972 ,  0.4800813 ,  1.1263399 ,
       -0.9571767 ,  1.5167176 ,  2.4747825 , -2.232404  ,  2.8815398 ,
       -3.84935   ,  2.4273005 , -0.0388324 ,  0.07375836, -1.9437244 ,
       -2.5345845 ,  0.32627085, -0.52031565, -1.6569221 , -1.452766  ,
       -4.066375  , -2.7125504 , -2.0605338 ,  2.3531666 , -2.2015738 ,
       -1.3226348 , -0.3980228 ,  0.32416967, -2.3192768 ,  3.910695  ,
       -1.1025765 , -0.64029545, -1.7138597 , -3.528511  ,  1.1143405 ,
       -0.11756355,  0.28521106,  2.1705024 ,  0.03962039,  2.611368  ,
       -0.15945473, -1.0234438 ,  3.7989838 ,  0.32224187,  2.04

In [95]:
model.wv.similarity("voiture", "carrosse")

0.24132355

In [101]:
model.wv.most_similar("boucher", topn=10)

[('tailleur', 0.6623472571372986),
 ('boulanger', 0.6560649275779724),
 ('cabaretier', 0.6487619280815125),
 ('menuisier', 0.6416441798210144),
 ('serrurier', 0.6407414078712463),
 ('charcutier', 0.6258613467216492),
 ('tapissier', 0.614556074142456),
 ('terrassier', 0.6109156012535095),
 ('cordonnier', 0.6100437641143799),
 ('fabricant', 0.6039721965789795)]

In [109]:
print(model.wv.most_similar(positive=['homme', 'roi'], negative=['femme']))

[('peuple', 0.5006588101387024), ('souverain', 0.49426090717315674), ('parlement', 0.489849716424942), ('pouvoir_legislatif', 0.48538702726364136), ('prince', 0.48223361372947693), ('citoyen', 0.4811207354068756), ('magistrat_communal', 0.46513354778289795), ('pouvoir_civil', 0.4647432267665863), ('gouvernement', 0.4601726233959198), ('soldat', 0.4586349129676819)]


In [111]:
print(model.wv.most_similar(positive=['homme', 'bourgmestre'], negative=['femme']))

[('collegue', 0.4746754467487335), ('defenseur', 0.45523303747177124), ('scrupule', 0.44064831733703613), ('conseiller', 0.43790969252586365), ('temoignage', 0.43457233905792236), ('sentiment', 0.4340687692165375), ('orateur', 0.4326702356338501), ('geste', 0.43116092681884766), ('ministre', 0.42899051308631897), ('pouvoir_legislatif', 0.4280470013618469)]


In [112]:
print(model.wv.most_similar(positive=['bourgmestre', 'bruxelles'], negative=['echevin']))

[('berne', 0.39495036005973816), ('nivelles', 0.3824808895587921), ('huy', 0.38201335072517395), ('prague', 0.37479788064956665), ('zurich', 0.3697776198387146), ('liege', 0.3689724802970886), ('turnhout', 0.3591597080230713), ('bruxelle', 0.3570358157157898), ('gand', 0.3554554581642151), ('verviers', 0.3481411039829254)]


### Modèle 7 : 300 dimensions et élargissement de la fenêtre à 40

In [ ]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=300, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=40, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

In [ ]:
outfile = f"../data/bulletins_7.model"
model.save(outfile)

In [ ]:
model = Word2Vec.load("../data/bulletins_7.model")

In [ ]:
model.wv["bruxelles"]

In [ ]:
model.wv.similarity("boucher", "boulanger")

In [ ]:
model.wv.similarity("voiture", "carrosse")

In [ ]:
model.wv.most_similar("bruxelles", topn=10)

In [ ]:
model.wv.most_similar("boucher", topn=10)

In [ ]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

In [ ]:
print(model.wv.most_similar(positive=['homme', 'roi'], negative=['femme']))

In [ ]:
print(model.wv.most_similar(positive=['homme', 'bourgmestre'], negative=['femme']))

In [ ]:
print(model.wv.most_similar(positive=['bourgmestre', 'bruxelles'], negative=['echevin']))

In [ ]:
print(model.wv.most_similar(positive=['bruxelles', 'paris'], negative=['belgique']))